# Contrib.layers (Feature_column)

### layers.real_valued_column = Handles the numerical data
### layers.sparse_column_with_keys() = Handles the conversion of categorical values 
### layers.sparse_column_with_hash_bucket() = Handles the conversion of categorical features for which you don’t know all possible values 

### layers.crossed_column() = Sets up feature crosses (interactions) 
### layers.bucketized_column() = Turns a continuous column into a categorical column 

In [3]:
import pandas as pd 
import numpy as np

N = 10000
weight = np.random.randn(N)*5+70 
spec_id = np.random.randint(0,3,N) 
bias = [0.9,1,1.1] 
height = np.array([weight[i]/100 + bias[b] for i,b in enumerate(spec_id)]) 


spec_name = ['Goblin','Human','ManBears'] 
spec = [spec_name[s] for s in spec_id] 

In [14]:
print(weight.shape)
print(height.shape)
print(np.array(spec).shape)

(10000,)
(10000,)
(10000,)


In [8]:
#We use the Pandas library to have the data represented as a data frame (table), 
#so that we can conveniently access each of its columns:

df = pd.DataFrame({'Species':spec,'Weight':weight,'Height':height})
df

,Species,Weight,Height
0,Human,66.691582,1.666916
1,ManBears,72.902635,1.829026
2,ManBears,65.321091,1.753211
3,Goblin,58.831351,1.488314
4,Goblin,76.520349,1.665203
5,ManBears,69.594103,1.795941
6,Goblin,71.352085,1.613521
7,Goblin,64.987183,1.549872
8,Human,67.309308,1.673093
9,ManBears,75.675694,1.856757


# Defien Features_Column 
# Preprocessing

In [109]:
from tensorflow.contrib import learn
import tensorflow as tf


#Working On Two Features
Weight = layers.real_valued_column("Weight")  #numerical data
Species = layers.sparse_column_with_keys(column_name="Species", keys=['Goblin','Human','ManBears'])  #categorical data

print(Weight)
print(Species)

_RealValuedColumn(column_name='Weight', dimension=1, default_value=None, dtype=tf.float32, normalizer=None)
_SparseColumnKeys(column_name='Species', is_integerized=False, bucket_size=None, lookup_config=_SparseIdLookupConfig(vocabulary_file=None, keys=('Goblin', 'Human', 'ManBears'), num_oov_buckets=0, vocab_size=3, default_value=-1), combiner='sum', dtype=tf.string)


## input_fn(df)

#### (**The function gets both predictors and target data in their native form (Pandas data frame, NumPy array, list, etc.) as input, and returns a dictionary of tensors)

####  A dictionary feature_cols, and then stores the values of each column in the data frame as a Tensor for the corresponding key.

#### It then returns that dictionary and the target variable as a Tensor. The keys have to match the names we used to define our FeatureColumns:


In [110]:
#We start by specifying the nature of each feature. 
#For Weight we use the following FeatureColumn command, indicating that it’s a continuous variable:

from tensorflow.contrib import layers 

def input_fn(df):    
    feature_cols = {}    
    feature_cols['Weight'] = tf.constant(df['Weight'].values)        
    feature_cols['Species'] =  tf.SparseTensor(indices=[[i, 0] for i in range(df['Species'].size)],
                                               values=df['Species'].values, 
                                               dense_shape=[df['Species'].size, 1])                        
    
    labels = tf.constant(df['Height'].values)
    return feature_cols, labels 


## Linear Regressor (Estimator)

In [64]:
reg = learn.LinearRegressor(feature_columns=[Weight,Species]) 

reg.fit(input_fn=lambda:input_fn(df), steps=50000)


Instructions for updating:
Please switch to tf.contrib.estimator.*_head.
Instructions for updating:
Please replace uses of any Estimator from tf.contrib.learn with an Estimator from tf.estimator.*
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E70E5FF470>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5

INFO:tensorflow:global_step/sec: 190.115
INFO:tensorflow:loss = 0.00090036343, step = 5400 (0.526 sec)
INFO:tensorflow:global_step/sec: 204.081
INFO:tensorflow:loss = 0.00087331916, step = 5500 (0.490 sec)
INFO:tensorflow:global_step/sec: 212.767
INFO:tensorflow:loss = 0.0008470871, step = 5600 (0.469 sec)
INFO:tensorflow:global_step/sec: 212.314
INFO:tensorflow:loss = 0.00082164316, step = 5700 (0.472 sec)
INFO:tensorflow:global_step/sec: 209.204
INFO:tensorflow:loss = 0.000796963, step = 5800 (0.478 sec)
INFO:tensorflow:global_step/sec: 213.676
INFO:tensorflow:loss = 0.00077302445, step = 5900 (0.468 sec)
INFO:tensorflow:global_step/sec: 215.517
INFO:tensorflow:loss = 0.00074980507, step = 6000 (0.464 sec)
INFO:tensorflow:global_step/sec: 215.053
INFO:tensorflow:loss = 0.00072728336, step = 6100 (0.465 sec)
INFO:tensorflow:global_step/sec: 209.206
INFO:tensorflow:loss = 0.0007054374, step = 6200 (0.478 sec)
INFO:tensorflow:global_step/sec: 232.018
INFO:tensorflow:loss = 0.0006842484,

INFO:tensorflow:global_step/sec: 208.332
INFO:tensorflow:loss = 7.8495905e-05, step = 13400 (0.479 sec)
INFO:tensorflow:global_step/sec: 224.216
INFO:tensorflow:loss = 7.613814e-05, step = 13500 (0.446 sec)
INFO:tensorflow:global_step/sec: 224.719
INFO:tensorflow:loss = 7.385133e-05, step = 13600 (0.445 sec)
INFO:tensorflow:global_step/sec: 226.244
INFO:tensorflow:loss = 7.16332e-05, step = 13700 (0.442 sec)
INFO:tensorflow:global_step/sec: 225.225
INFO:tensorflow:loss = 6.9481466e-05, step = 13800 (0.444 sec)
INFO:tensorflow:global_step/sec: 226.245
INFO:tensorflow:loss = 6.739451e-05, step = 13900 (0.442 sec)
INFO:tensorflow:global_step/sec: 226.244
INFO:tensorflow:loss = 6.5370165e-05, step = 14000 (0.442 sec)
INFO:tensorflow:global_step/sec: 226.245
INFO:tensorflow:loss = 6.340672e-05, step = 14100 (0.443 sec)
INFO:tensorflow:global_step/sec: 224.719
INFO:tensorflow:loss = 6.150224e-05, step = 14200 (0.444 sec)
INFO:tensorflow:global_step/sec: 224.215
INFO:tensorflow:loss = 5.96551

INFO:tensorflow:loss = 7.0594756e-06, step = 21300 (0.467 sec)
INFO:tensorflow:global_step/sec: 214.121
INFO:tensorflow:loss = 6.8476497e-06, step = 21400 (0.468 sec)
INFO:tensorflow:global_step/sec: 203.677
INFO:tensorflow:loss = 6.641978e-06, step = 21500 (0.491 sec)
INFO:tensorflow:global_step/sec: 201.196
INFO:tensorflow:loss = 6.442731e-06, step = 21600 (0.496 sec)
INFO:tensorflow:global_step/sec: 209.656
INFO:tensorflow:loss = 6.2493154e-06, step = 21700 (0.478 sec)
INFO:tensorflow:global_step/sec: 207.451
INFO:tensorflow:loss = 6.061764e-06, step = 21800 (0.481 sec)
INFO:tensorflow:global_step/sec: 207.057
INFO:tensorflow:loss = 5.8799033e-06, step = 21900 (0.484 sec)
INFO:tensorflow:global_step/sec: 202.84
INFO:tensorflow:loss = 5.7033444e-06, step = 22000 (0.493 sec)
INFO:tensorflow:global_step/sec: 205.338
INFO:tensorflow:loss = 5.5322334e-06, step = 22100 (0.487 sec)
INFO:tensorflow:global_step/sec: 206.613
INFO:tensorflow:loss = 5.3662843e-06, step = 22200 (0.484 sec)
INFO:

INFO:tensorflow:global_step/sec: 219.774
INFO:tensorflow:loss = 6.1741247e-07, step = 29300 (0.456 sec)
INFO:tensorflow:global_step/sec: 217.869
INFO:tensorflow:loss = 5.989123e-07, step = 29400 (0.458 sec)
INFO:tensorflow:global_step/sec: 216.922
INFO:tensorflow:loss = 5.808713e-07, step = 29500 (0.462 sec)
INFO:tensorflow:global_step/sec: 222.214
INFO:tensorflow:loss = 5.6338615e-07, step = 29600 (0.450 sec)
INFO:tensorflow:global_step/sec: 219.788
INFO:tensorflow:loss = 5.4644676e-07, step = 29700 (0.454 sec)
INFO:tensorflow:global_step/sec: 212.766
INFO:tensorflow:loss = 5.3014463e-07, step = 29800 (0.471 sec)
INFO:tensorflow:global_step/sec: 222.221
INFO:tensorflow:loss = 5.143551e-07, step = 29900 (0.450 sec)
INFO:tensorflow:global_step/sec: 218.34
INFO:tensorflow:loss = 4.989441e-07, step = 30000 (0.458 sec)
INFO:tensorflow:global_step/sec: 210.527
INFO:tensorflow:loss = 4.83773e-07, step = 30100 (0.474 sec)
INFO:tensorflow:global_step/sec: 209.644
INFO:tensorflow:loss = 4.69134

INFO:tensorflow:loss = 5.5782586e-08, step = 37200 (0.476 sec)
INFO:tensorflow:global_step/sec: 220.256
INFO:tensorflow:loss = 5.411295e-08, step = 37300 (0.454 sec)
INFO:tensorflow:global_step/sec: 221.729
INFO:tensorflow:loss = 5.2480406e-08, step = 37400 (0.452 sec)
INFO:tensorflow:global_step/sec: 219.789
INFO:tensorflow:loss = 5.0928957e-08, step = 37500 (0.456 sec)
INFO:tensorflow:global_step/sec: 210.085
INFO:tensorflow:loss = 4.947061e-08, step = 37600 (0.475 sec)
INFO:tensorflow:global_step/sec: 224.215
INFO:tensorflow:loss = 4.8031502e-08, step = 37700 (0.445 sec)
INFO:tensorflow:global_step/sec: 224.215
INFO:tensorflow:loss = 4.6614478e-08, step = 37800 (0.447 sec)
INFO:tensorflow:global_step/sec: 221.724
INFO:tensorflow:loss = 4.5217416e-08, step = 37900 (0.450 sec)
INFO:tensorflow:global_step/sec: 223.22
INFO:tensorflow:loss = 4.384103e-08, step = 38000 (0.449 sec)
INFO:tensorflow:global_step/sec: 221.728
INFO:tensorflow:loss = 4.2487496e-08, step = 38100 (0.451 sec)
INFO:

INFO:tensorflow:global_step/sec: 222.717
INFO:tensorflow:loss = 4.969046e-09, step = 45200 (0.450 sec)
INFO:tensorflow:global_step/sec: 221.728
INFO:tensorflow:loss = 4.822794e-09, step = 45300 (0.451 sec)
INFO:tensorflow:global_step/sec: 216.449
INFO:tensorflow:loss = 4.6779367e-09, step = 45400 (0.462 sec)
INFO:tensorflow:global_step/sec: 216.921
INFO:tensorflow:loss = 4.5360125e-09, step = 45500 (0.461 sec)
INFO:tensorflow:global_step/sec: 218.341
INFO:tensorflow:loss = 4.397473e-09, step = 45600 (0.458 sec)
INFO:tensorflow:global_step/sec: 215.517
INFO:tensorflow:loss = 4.2606283e-09, step = 45700 (0.463 sec)
INFO:tensorflow:global_step/sec: 212.766
INFO:tensorflow:loss = 4.126177e-09, step = 45800 (0.470 sec)
INFO:tensorflow:global_step/sec: 205.762
INFO:tensorflow:loss = 3.9936423e-09, step = 45900 (0.487 sec)
INFO:tensorflow:global_step/sec: 217.391
INFO:tensorflow:loss = 3.8682804e-09, step = 46000 (0.460 sec)
INFO:tensorflow:global_step/sec: 206.611
INFO:tensorflow:loss = 3.76

LinearRegressor(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x000001E70E5FEEF0>, 'feature_columns': [_RealValuedColumn(column_name='Weight', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _SparseColumnKeys(column_name='Species', is_integerized=False, bucket_size=None, lookup_config=_SparseIdLookupConfig(vocabulary_file=None, keys=('Goblin', 'Human', 'ManBears'), num_oov_buckets=0, vocab_size=3, default_value=-1), combiner='sum', dtype=tf.string)], 'optimizer': None, 'gradient_clip_norm': None, 'joint_weights': False})

In [104]:
MSE = reg.evaluate(input_fn=lambda:input_fn(df), steps=1)
MSE

INFO:tensorflow:Starting evaluation at 2018-12-24-03:52:10
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\DELL\AppData\Local\Temp\tmpbjc2xndh\model.ckpt-50000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2018-12-24-03:52:10
INFO:tensorflow:Saving dict for global step 50000: global_step = 50000, loss = 1.0628324e-09


{'loss': 1.0628324e-09, 'global_step': 50000}